# Bot for data collection in Lattes

## Necessary libraries

In [ ]:
try:
    import re
    import threading
    import pandas as pd
    import time as time
    import database as db
    from bs4 import BeautifulSoup
    from unidecode import unidecode
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from selenium.webdriver.common.keys import Keys 
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
except Exception as e:
    print(e)

## Bot

In [ ]:
class BotLattes:
    """
    
    Bot para coleta de dados no Lattes
    Limitações:
        - Formação de busca: Doutorado, Mestrado, Todos.
        - Tipos de trabalhos coletados: Artigos completos publicados em periódicos.
        - Participantes do artigo: Apenas o responsável pelo currículo.
    
    """
    def __init__(self, formacao, inicio=0, limitePaginas=None, curriculo=1, nome="Bot", debug=False):
        """
        
        Args:
            formacao [string] : Nível de formação para coleta dos dados.
            inicio [int, optional] : Página a qual será iniciada a coleta de currículos. Defaults to 0.
            limitePaginas [int, optional] : Quantidade máxima de páginas a serem coletadas. Defaults to None.
            curriculo [int, optional] : Currículo inicial da página onde o bot inicia a coleta. Defaults to 1.
            nome [string, optional] : Nome do bot para identificação. Defaults to Bot.
        
        """
        
        print(f"----- Start Bot {nome} - {inicio} <-> {limitePaginas} -----")
        
        #Váriaveis de controle
        self.nome = nome
        self.limite = None
        self.inicio = inicio
        self.inicioP = inicio
        self.fim = limitePaginas
        self.formacao = formacao
        self.debug = debug
        self.pagina = inicio

        #Váriaveis de execução
        self.data = db.DataBase()
        self.init = time.perf_counter()
        self.op = Options()
        self.op.headless = True
        self.delay_max = 200
        self.delimiters = ".", ",", ";", ":"
        self.regexPattern = '|'.join(map(re.escape, self.delimiters))
        
        if self.debug:
            print(f"[Bot {self.nome}]Inicio")
        self.getData(curriculo=curriculo)
        
        print(f"----- End {self.nome} - {self.inicioP} <-> {self.fim} || Tempo de execução: {time.perf_counter()-self.init:.2f} ----\n")
        
    def close(self):
        """
        
        Finaliza o bot e armazena os dados coletados.
        Em caso de erro, finaliza o bot e imprime a mensagem:
        Bot {nome do bot} - Error:\n{erro ocorrido}

        """
        if self.debug:
            print(f"[Bot {self.nome}]Close")
            
        try:            
            self.data.saveInFile(2018)
            self.data.saveInFile(2019)
            self.data.saveInFile(2020)
            self.data.saveInFile(2021)
            self.data.connectionfile.commit()
            self.data.connection.close()
        except Exception as e:
            print(f"[Bot {self.nome}]Close - Error {e}")
            return
        
    def getData(self, curriculo=1, **kwargs):
        """
        
        Controle da coleta de dados.

        Args:
            curriculo [int, optional] : Currículo inicial da página onde o bot inicia a coleta. Defaults to 1.
        
        """
        
        if self.debug:
            print(f"[Bot {self.nome}]getData")
        
        try:
            self.navegador = webdriver.Chrome(options=self.op)
            
            self.navegador.get("http://buscatextual.cnpq.br/buscatextual/busca.do")
        except Exception as e:
            if self.debug:
                print(f"[Bot {self.nome}]getData - Erro {e}")
            self.getData(curriculo=curriculo)
        
        try:
            self.inicio = kwargs["inicio"]
        except:
            pass
        
        self.parametros()
        
        time.sleep(1)
        
        self.passadas = []

        self.janelaAtual = self.navegador.current_window_handle
        self.passadas.append(self.navegador.window_handles)

        #Variaveis de controle da paginação
        self.pag = 2
        self.troca = True

        self.limite_busca()
        
        self.inicio_coleta()
        
        while True:
            if self.troca:
                self.trocar_pagina()
            self.troca = False
            end = self.selecionar_curriculo(curriculo)
            
            if end[0] == True:
                break
            
            self.data.connection.commit()

            #Limitador de pagínas
            if self.pagina == self.fim:
                break
            
            self.pagina += 1
            curriculo = 1
            self.trocar_pagina()

        self.navegador.quit()
        if self.pagina == self.fim or self.pagina == self.limite:
            self.close()
        else:
            self.getData(curriculo=end[1], **{"inicio": self.pagina})
        
    def parametros(self):
        """
        
        Define os parâmetros de pesquisa.
        
        """
        if self.debug:
            print(f"[Bot {self.nome}]Parametros")
        try:            
            WebDriverWait(self.navegador, self.delay_max).until(EC.presence_of_element_located((By.XPATH, '//*[@id="tit_simples"]/a')))
            self.navegador.find_element_by_xpath('//*[@id="tit_simples"]/a').click()

            #Tags de pesquisa
            if self.formacao != "doutorado":
                WebDriverWait(self.navegador, self.delay_max).until(EC.presence_of_element_located((By.XPATH, '//*[@id="buscarDemaisAvancada"]')))
                self.navegador.find_element_by_xpath('//*[@id="buscarDemaisAvancada"]').send_keys(Keys.SPACE) #Buscar demais pesquisadores
                self.navegador.find_element_by_xpath('//*[@id="buscarDoutoresAvancada"]').send_keys(Keys.SPACE) #Buscar demais pesquisadores
                
                if self.formacao == "mestrado":
                    WebDriverWait(self.navegador, self.delay_max).until(EC.presence_of_element_located((By.XPATH, '//*[@id="filtro2"]')))
                    self.navegador.find_element_by_xpath('//*[@id="filtro2"]').click()
                    time.sleep(1)
                    self.navegador.find_element_by_xpath('//*[@id="nivelFormacao"]').click()
                    self.navegador.find_element_by_xpath('//*[@id="nivelFormacao"]').send_keys(Keys.DOWN)
                    self.navegador.find_element_by_xpath('//*[@id="nivelFormacao"]').send_keys(Keys.DOWN)
                    self.navegador.find_element_by_xpath('//*[@id="nivelFormacao"]').send_keys(Keys.DOWN)
                    self.navegador.find_element_by_xpath('//*[@id="nivelFormacao"]').send_keys(Keys.ENTER)
                    time.sleep(1)
                    #WebDriverWait(navegador, self.delay_max).until(EC.presence_of_element_located((By.XPATH, '//*[@id="preencheCategoriaNivelBolsa"]')))
                    self.navegador.find_element_by_xpath('/html/body/form/div/div[4]/div/div/div/div[11]/div[2]/div/div/div/a[1]').click()

            WebDriverWait(self.navegador, self.delay_max).until(EC.presence_of_element_located((By.XPATH, '//*[@id="buscarEstrangeirosAvancada"]')))
            self.navegador.find_element_by_xpath('//*[@id="buscarEstrangeirosAvancada"]').click()

            #Caixa de texto
            buscar = self.navegador.find_element_by_xpath('//*[@id="busca_avancada"]/div/div[2]/div/fieldset/textarea')
            buscar.send_keys("(2018 OR 2019 OR 2020 OR 2021)")
            buscar.send_keys(Keys.ENTER)
        except Exception as e:
            if self.debug:
                print(f"[Bot {self.nome}]Parametros - Erro {e}",end="")
            self.parametros()
            
    def limite_busca(self):
        """
        
        Verifica os limites de coleta.
        
        """
        if self.debug:
            print(f"[Bot {self.nome}]Limite_busca")
        try:
            WebDriverWait(self.navegador, self.delay_max).until(EC.presence_of_element_located((By.XPATH, '/html/body/form/div/div[4]/div/div/div/div[3]/div/div[1]/b[1]')))
            self.limite = int(self.navegador.find_element_by_xpath('/html/body/form/div/div[4]/div/div/div/div[3]/div/div[1]/b[1]').get_attribute("innerHTML").splitlines()[0])
            self.limite = int(self.limite/10 if self.limite%10==0 else (self.limite/10)+1)
        except Exception as e:
            if self.debug:
                print(f"[Bot {self.nome}]Limite_busca - Erro {e}")
            if self.limite == None:
                self.limite_busca()
            else:
                pass
    
    def inicio_coleta(self):
        """
        
        Seleciona a página de início da coleta.
        
        """
        if self.debug:
            print(f"[Bot {self.nome}]Inicio_coleta")
        if self.inicio!=0:
            try:
                WebDriverWait(self.navegador, self.delay_max).until(EC.presence_of_element_located((By.XPATH, f'/html/body/form/div/div[4]/div/div/div/div[3]/div/div[4]/a[{self.pag}]')))
                self.navegador.find_element_by_xpath(f'/html/body/form/div/div[4]/div/div/div/div[3]/div/div[4]/a[{self.pag}]').click()
                pagina = self.inicio
                link = self.navegador.current_url.split(";")
                link[0] = link[0].replace(str(10), str((pagina-1)*10))
                link = link[0]+";"+link[1]

                if self.inicio<11:
                    self.pag = self.inicio+1
                elif self.inicio%10==0:
                    self.pag = 13
                else:
                    div = "1"+("0"*(len(str(self.inicio))-1))
                    self.pag = int((self.inicio+1)/int(div))+2
                self.navegador.get(link)
                self.troca = False
                self.inicio = 0
            except Exception as e:
                if self.debug:
                    print(f"[Bot {self.nome}]Inicio_coleta - Erro {e}")
                self.inicio_coleta(self.inicio)
    
    def trocar_pagina(self):
        """
        
        Faz a troca de páginas.
        
        """
        if self.debug:
            print(f"[Bot {self.nome}]Troca_pagina")
        try:
            if self.pagina!=1 and self.pag<=11 and self.pagina<=11:
                WebDriverWait(self.navegador, self.delay_max).until(EC.presence_of_element_located((By.XPATH, f'/html/body/form/div/div[4]/div/div/div/div[3]/div/div[4]/a[{self.pag}]')))
                self.navegador.find_element_by_xpath(f'/html/body/form/div/div[4]/div/div/div/div[3]/div/div[4]/a[{self.pag}]').click()
                self.pag = self.pag+1 if self.pag<11 else 3

            if self.pagina>11 and self.pag>2:
                WebDriverWait(self.navegador, self.delay_max).until(EC.presence_of_element_located((By.XPATH, f'/html/body/form/div/div[4]/div/div/div/div[3]/div/div[4]/a[{self.pag}]')))
                self.navegador.find_element_by_xpath(f'/html/body/form/div/div[4]/div/div/div/div[3]/div/div[4]/a[{self.pag}]').click()
                self.pag = self.pag+1 if self.pag<13 else 3
        except Exception as e:
            if self.debug:
                print(f"[Bot {self.nome}]Troca_pagina - Erro {e}",end="")
    
    def selecionar_curriculo(self, curriculo):
        """
        
        Faz a seleção do currículo

        Args:
            curriculo [int]: currículo do início da pesquisa.

        Returns:
            list: lista com dois itens: 
                    Tipo bool sinalizando problema na coleta da página.
                    Tipo int representando o último currículo selecionado.
        """
        if self.debug:
            print(f"[Bot {self.nome}]Selecionar_curriculo")
        endFor = False
        i = curriculo
        while i < 11:
            try:
                #Selecionar curriculo
                WebDriverWait(self.navegador, self.delay_max).until(EC.presence_of_element_located((By.XPATH, f'//html/body/form/div/div[4]/div/div/div/div[3]/div/div[3]/ol/li[{i}]/b/a')))
                self.navegador.find_element_by_xpath(f'//html/body/form/div/div[4]/div/div/div/div[3]/div/div[3]/ol/li[{i}]/b/a').click()

                time.sleep(1)

                #Abrir curriculo
                WebDriverWait(self.navegador, self.delay_max).until(EC.presence_of_element_located((By.XPATH, '//*[@id="idbtnabrircurriculo"]')))
                self.navegador.find_element_by_xpath('//*[@id="idbtnabrircurriculo"]').click()

            except Exception as e:
                if self.debug:
                    print(f"[Bot {self.nome}]Selecionar_curriculo - Erro ", end="")
                print(e)
                endFor = True
                break

            time.sleep(2)
            
            i += self.trocar_janela()
            
            i += self.coleta()
            
            i+=1
        
        return endFor, i
    
    def trocar_janela(self):
        """
        
        Troca da janela de pesquisa para a janela com o currículo.

        Returns:
            [int]: Valor de representação de erro. 0 execução correta e -1 para ocorrência de erro.
        """
        if self.debug:
            print(f"[Bot {self.nome}]Troca_janela")
        try:
            for janela in self.navegador.window_handles:
                if janela not in self.passadas:
                    self.navegador.switch_to.window(janela)

        except Exception as e:
            if self.debug:
                print(f"[Bot {self.nome}]Troca_janela - Erro {e}")
            self.navegador.switch_to.window(self.janelaAtual)
            self.navegador.get(self.navegador.current_url)
            return -1
        
        return 0
    
    def coleta(self):
        """
        
        Coleta dos dados dentro do currículo.

        Returns:
            [int]: Valor de representação de erro. 0 execução correta e -1 para ocorrência de erro.
        """
        if self.debug:
            print(f"[Bot {self.nome}]Coleta")
        try:
            #Pegar dados
            WebDriverWait(self.navegador, self.delay_max).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[3]/div/div/div/div[1]/h2'))) 
            site = BeautifulSoup(self.navegador.page_source, 'html.parser')
            artigos2018 = []
            artigos2019 = []
            artigos2020 = []
            artigos2021 = []


            for artigo in site.findAll('div', attrs={'class':['artigo-completo']}):
                maior = max(re.split(self.regexPattern, artigo.text), key=len)
                
                if "2018" in artigo.text:
                    artigos2018.append(maior)
                    
                elif "2019" in artigo.text:
                    artigos2019.append(maior)
                    
                elif "2020" in artigo.text:
                    artigos2020.append(maior)
                    
                elif "2021" in artigo.text:
                    artigos2021.append(maior)

            nome = site.findAll('h2', attrs={'class':['nome']})[0].text
        except Exception as e:
            time.sleep(1)
            if self.debug:
                print(f"[Bot {self.nome}]Coleta - Erro ", end="")
            print(e)
            return -1

        #Inserir no banco de dados
        if artigos2018 != []:
            self.data.insertData(nome, self.formacao, artigos2018, 2018)
        if artigos2019 != []:
            self.data.insertData(nome, self.formacao, artigos2019, 2019)
        if artigos2020 != []:
            self.data.insertData(nome, self.formacao, artigos2020, 2020)
        if artigos2021 != []:
            self.data.insertData(nome, self.formacao, artigos2021, 2021)
        
        self.navegador.close()

        time.sleep(1)

        self.navegador.switch_to.window(self.janelaAtual)

        #Fechar curriculo
        WebDriverWait(self.navegador, self.delay_max).until(EC.presence_of_element_located((By.XPATH, '//*[@id="idbtnfechar"]')))
        self.navegador.find_element_by_xpath('//*[@id="idbtnfechar"]').click()
        
        return 0

## Collect

In [ ]:
def gerarDivs(tamanho, fim, inicio):
    """
    Gerador de divisão de páginas para o bot de coleta.

    Args:
        tamanho [int]: Tamanho de cada bloco de páginas.
        fim [int]: Última pagina para a coleta.
        inicio [int]: Página de inicio da coleta.

    Returns:
        [list]: Lista com os blocos de páginas a serem coletados,
        cada item com a seguinte estrutura:
            [Bloco iniciado (bool), inicio do bloco (int), fim do bloco (int)]
    """
    listDivs = []
    for i in range(inicio, fim+1, tamanho):
        listDivs.append([False, i, i+(tamanho-1)])
    return listDivs

In [ ]:
class Controle:
    """
    
    Controle dos bots para a coleta de dados.
    
    """
    def __init__(self, quantidade, inicio, fim, tamanho=50, **kwargs):
        """

        Args:
            quantidade [int]: Quantidade de bots.
            inicio [int]: Página de inicio da coleta
            fim [int]: Página de fim da coleta
            tamanho [int, optional]: Tamanho de cada bloco de páginas. Defaults to 50.

        Raises:
            Exception: Nível de formação é obrigatório.
        
        """
        self.divs = gerarDivs(inicio=inicio, fim=fim, tamanho=tamanho)
        self.pos = 0
        self.botsAtivos = []
        self.bots = []
        self.quant = quantidade
        try:
            kwargs["formacao"]
            self.kwargs = kwargs
        except:
            raise Exception("Nível de formação não definido.")
        
        print("----- Gerar bot -----")
        self.gerarBot()
        
        print("----- Coleta -----")
        self.coleta()
    
    def gerarBot(self):
        """
        
        Gerar Bots.
        
        """
        for i in range(self.quant):
            self.bots.append(threading.Thread(target=lambda:BotLattes(self.kwargs["formacao"], debug=True, inicio=self.divs[self.pos][1], limitePaginas=self.divs[self.pos][2], nome=f"{i+1}")))
            self.botsAtivos.append(True)
            self.pos += 1
        self.pos = 0
        
    def coleta(self):
        """
        
        Controla a execução de cada bot.
        
        """
        while(self.botsAtivos):
            if self.pos < len(self.divs):
                for bot in range(len(self.bots)):
                    if not self.bots[bot].is_alive() and self.botsAtivos[bot]:
                        if not self.divs[self.pos][0]:
                            self.bots[bot] = threading.Thread(target=lambda:BotLattes(self.kwargs["formacao"], debug=True, inicio=self.divs[self.pos][1], limitePaginas=self.divs[self.pos][2], nome=f"{bot+1}"))
                            self.bots[bot].start()
                            self.divs[self.pos][0] = True
                            self.pos += 1
                        else:
                            self.botsAtivos[bot] = False
            else:
                break
                        
    

In [ ]:
Controle(5, 1, 25, 4, **{"formacao":"doutorado"})

## Analysis of results

In [ ]:
data = db.DataBase()

In [ ]:
df18 = data.selectData(2018)

In [ ]:
df19 = data.selectData(2019)

In [ ]:
df20 = data.selectData(2020)

In [ ]:
df21 = data.selectData(2021)

In [ ]:
df = pd.DataFrame(columns=list(df18.columns))
df = pd.merge(df, df18, how='outer')
df = pd.merge(df, df19, how='outer')
df = pd.merge(df, df20, how='outer')
df = pd.merge(df, df21, how='outer')

In [ ]:
df